# EliminationTree

An `EliminationTree` represents the computational structure of sequential variable elimination (like Gaussian elimination) on a `FactorGraph` given a specific `Ordering`.

Each node in the tree corresponds to a variable being eliminated. The children of a node represent variables that were eliminated earlier and produced factors involving the parent variable. The factors originally involving the variable at a node are stored at that node.

Eliminating an `EliminationTree` yields a `BayesNet`.

While fundamental to the theory, direct manipulation of `EliminationTree` objects in Python is less common than using the `eliminateSequential` method on a `FactorGraph`, which uses the `EliminationTree` internally.

<a href="https://colab.research.google.com/github/borglab/gtsam/blob/develop/gtsam/inference/doc/EliminationTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install gtsam

In [2]:
import gtsam
import numpy as np

# EliminationTree is templated, need concrete types
from gtsam import GaussianFactorGraph, Ordering, GaussianEliminationTree, GaussianBayesNet
from gtsam import symbol_shorthand

X = symbol_shorthand.X
L = symbol_shorthand.L

ImportError: cannot import name 'GaussianEliminationTree' from 'gtsam' (c:\Users\porte\miniconda3\envs\gtsam\Lib\site-packages\gtsam\__init__.py)

## Creating an EliminationTree

An `EliminationTree` is constructed from a `FactorGraph` and an `Ordering`.

In [ ]:
# Create a graph (same as BayesTree example)
graph = GaussianFactorGraph()
model = gtsam.noiseModel.Isotropic.Sigma(1, 1.0)
graph.add(X(0), -np.eye(1), np.zeros(1), model)
graph.add(X(0), -np.eye(1), X(1), np.eye(1), np.zeros(1), model)
graph.add(X(1), -np.eye(1), X(2), np.eye(1), np.zeros(1), model)
graph.add(L(1), -np.eye(1), X(0), np.eye(1), np.zeros(1), model)
graph.add(L(1), -np.eye(1), X(1), np.eye(1), np.zeros(1), model)
graph.add(L(2), -np.eye(1), X(1), np.eye(1), np.zeros(1), model)
graph.add(L(2), -np.eye(1), X(2), np.eye(1), np.zeros(1), model)

# Define an ordering
ordering = Ordering([X(0), L(1), X(1), L(2), X(2)])

# Construct the Elimination Tree
elimination_tree = GaussianEliminationTree(graph, ordering)

elimination_tree.print("Elimination Tree: ")

Elimination Tree: 
Root(s):
Node (x2)
JacobianFactor(keys = [8070450532247928833; 8070450532247928834], A[0] = [ -1  1 ], b = [ 0 ], model = diagonal sigmas [1])
JacobianFactor(keys = [7783684379976990721; 8070450532247928834], A[0] = [ -1  1 ], b = [ 0 ], model = diagonal sigmas [1])
  Children:
  Node (l2)
  JacobianFactor(keys = [7783684379976990721; 8070450532247928833], A[0] = [ -1  1 ], b = [ 0 ], model = diagonal sigmas [1])
    Children:
    Node (x1)
    JacobianFactor(keys = [8070450532247928832; 8070450532247928833], A[0] = [ -1  1 ], b = [ 0 ], model = diagonal sigmas [1])
    JacobianFactor(keys = [7783684379976990720; 8070450532247928833], A[0] = [ -1  1 ], b = [ 0 ], model = diagonal sigmas [1])
      Children:
      Node (l1)
      JacobianFactor(keys = [7783684379976990720; 8070450532247928832], A[0] = [ -1  1 ], b = [ 0 ], model = diagonal sigmas [1])
        Children:
        Node (x0)
        JacobianFactor(keys = [8070450532247928832], Z = [ -1 ], b = [ 0 ], model 

## Elimination

The primary use of an `EliminationTree` is to perform sequential elimination to produce a `BayesNet`.

In [ ]:
# The eliminate function needs to be specified (e.g., EliminateGaussian)
# In Python, this is usually handled internally by graph.eliminateSequential
# but the C++ EliminationTree has an eliminate method.

# Let's call the graph's eliminateSequential which uses the tree internally
bayes_net, remaining_graph = graph.eliminatePartialSequential(ordering)

print("BayesNet from EliminationTree:")
bayes_net.print()

BayesNet from EliminationTree: size 5
Conditional 0: GaussianConditional( P(x0 | l1) = dx0 - R*dl1 - d), R = [ 0.5 ], d = [ 0 ], sigmas = [ 0.866025 ])
Conditional 1: GaussianConditional( P(l1 | x1) = dl1 - R*dx1 - d), R = [ 0.5 ], d = [ 0 ], sigmas = [ 0.866025 ])
Conditional 2: GaussianConditional( P(x1 | l2, x2) = dx1 - R1*dl2 - R2*dx2 - d), R1 = [ 0.333333 ], R2 = [ 0.333333 ], d = [ 0 ], sigmas = [ 0.745356 ])
Conditional 3: GaussianConditional( P(l2 | x2) = dl2 - R*dx2 - d), R = [ 0.5 ], d = [ 0 ], sigmas = [ 0.866025 ])
Conditional 4: GaussianConditional( P(x2) = dx2 - d), d = [ 0 ], sigmas = [ 0.774597 ])

